In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from playwright.async_api import async_playwright
import re
from random import randint
import time

In [5]:
from requests_cache import CachedSession, FileCache
session = CachedSession('http_cache', backend='filesystem')

#### Get the CSV with all NYT Bestsellers

In [6]:
# load the csv with the NYT Bestsellers

df_nyt = pd.read_csv('nyt_raw.csv')

# keeping only unique title and author to avoid multiple scrapes for the same book

df_nyt_sub= df_nyt[['book', 'author1']]
df_nyt_unique = df_nyt_sub.drop_duplicates()

df_nyt_unique.reset_index(inplace=True)

df_nyt_unique.drop(columns= 'index', inplace=True)

df_nyt_unique.to_csv('df_nyt_unique.csv')

/var/folders/bp/pstyf_r14hx35s64y_db5pq40000gn/T/ipykernel_80840/649660456.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nyt_unique.drop(columns= 'index', inplace=True)


#### Write a loop to pick up books from csv rows and inserting book title in params 
(using cookies to keep sign in information in each scrape)

In [15]:
all_results = []
for index, row in df_nyt_unique.iterrows(): 
    book_title = row['book']
    author_name = row['author1']
    if (index + 1) % 20 == 0:
        time.sleep(29)
    else :
        time.sleep(randint(2, 6))
    try :
        cookies = {
            'ccsid': '742-3219006-7768950',
            '__qca': 'P0-2056977897-1700597220525',
            'lc-main': 'en_US',
            'ubid-main': '134-8959105-9465632',
            'csm-sid': '422-4199468-7443324',
            'session-id': '139-0527874-5342610',
            'session-id-time': '2332444182l',
            'session-token': '"YcXW7Jrp9Kdv2xKa/QBbi8yUBiSStEeiP28qLMGK2lSyGpW6Vw3XnxyZlwlCWSaBI23UAKLGg5mMq4mmjTTX5UMcJBP/l90fjKpsLaGzakLGDAaOiwusNYn+vfgZ/Ji7CiW7VI6WTJbTiBF+ktX/yjSNQ4si9UzZ1SHgTOg64mrm1HOLO7vvPVAkczr7/NrdNYMHm4GLcxETUjmEUyJHaBaWfvKLhflQHAYJK3X7Cvz7eQW7mrxpR5g/Z2bjmenos4nRiXFbFJ2gH5DpWR7mKNSM+yjI6VqdBq7LM4IoeVtaDzqpdTAlPHpDjMVc2EgfFeYGBLNWmZ6+Om+otBHfy9UZ+0k8Y+aI7yCqsEDM+7M="',
            'x-main': '"tlB5YO@Zn9Be1JOEOemZ3P13Hr@omAAwbvIMT7FJfMrlisFJpaLbLMo0vaTC0e9O"',
            'at-main': 'Atza|IwEBIDcy3eo7a_YNdQf_8OnGzwcey-G5RVHK7XfP5rWXif-4_Vi35NDoDynNj1XETaFcU4fwz2LR4aNWa7pNu4JaaOJyD3hWjg0TQM99HqHcdmQ7J8LLEkzbaY0Mr-w5fpbHcfNa5ex7oXPhsgAkO1SaBjRcwSW8Kn2LTONWfK_HCzEMd6yO04_V-rxxEDKo4CGs5f_qYC8N64UPEU5z8ujeLDuQR1yMOJq7x21AcB9nK0KEwMR1SFKqz8L-v6i6x8tF9PB5m8pC5ucLxADBO0u7Z7OV',
            'sess-at-main': '"ZgjCrkknePmuQiQKH0K2aBDhKl0Y1RS0IzdZ4GiPYrs="',
            'likely_has_account': 'true',
            'locale': 'en',
            '_session_id2': '3f6585ce11f778d9ddd2b236a4e40a86',
            '__gads': 'ID=ecab0ffeaa3f37cd:T=1701651617:RT=1702340873:S=ALNI_MaWm8wnJxWWL6w4EtAJDBKW8Mzakw',
            '__gpi': 'UID=00000ce2bab78f39:T=1701651617:RT=1702340873:S=ALNI_MbP8LYnaonBenczWFditmUQm_kvgA',
        }

        headers = {
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
            'Accept-Language': 'en-US,en;q=0.9',
            'Cache-Control': 'max-age=0',
            'Connection': 'keep-alive',
            # 'Cookie': 'ccsid=742-3219006-7768950; __qca=P0-2056977897-1700597220525; lc-main=en_US; ubid-main=134-8959105-9465632; csm-sid=422-4199468-7443324; session-id=139-0527874-5342610; session-id-time=2332444182l; session-token="YcXW7Jrp9Kdv2xKa/QBbi8yUBiSStEeiP28qLMGK2lSyGpW6Vw3XnxyZlwlCWSaBI23UAKLGg5mMq4mmjTTX5UMcJBP/l90fjKpsLaGzakLGDAaOiwusNYn+vfgZ/Ji7CiW7VI6WTJbTiBF+ktX/yjSNQ4si9UzZ1SHgTOg64mrm1HOLO7vvPVAkczr7/NrdNYMHm4GLcxETUjmEUyJHaBaWfvKLhflQHAYJK3X7Cvz7eQW7mrxpR5g/Z2bjmenos4nRiXFbFJ2gH5DpWR7mKNSM+yjI6VqdBq7LM4IoeVtaDzqpdTAlPHpDjMVc2EgfFeYGBLNWmZ6+Om+otBHfy9UZ+0k8Y+aI7yCqsEDM+7M="; x-main="tlB5YO@Zn9Be1JOEOemZ3P13Hr@omAAwbvIMT7FJfMrlisFJpaLbLMo0vaTC0e9O"; at-main=Atza|IwEBIDcy3eo7a_YNdQf_8OnGzwcey-G5RVHK7XfP5rWXif-4_Vi35NDoDynNj1XETaFcU4fwz2LR4aNWa7pNu4JaaOJyD3hWjg0TQM99HqHcdmQ7J8LLEkzbaY0Mr-w5fpbHcfNa5ex7oXPhsgAkO1SaBjRcwSW8Kn2LTONWfK_HCzEMd6yO04_V-rxxEDKo4CGs5f_qYC8N64UPEU5z8ujeLDuQR1yMOJq7x21AcB9nK0KEwMR1SFKqz8L-v6i6x8tF9PB5m8pC5ucLxADBO0u7Z7OV; sess-at-main="ZgjCrkknePmuQiQKH0K2aBDhKl0Y1RS0IzdZ4GiPYrs="; likely_has_account=true; locale=en; _session_id2=3f6585ce11f778d9ddd2b236a4e40a86; __gads=ID=ecab0ffeaa3f37cd:T=1701651617:RT=1702340873:S=ALNI_MaWm8wnJxWWL6w4EtAJDBKW8Mzakw; __gpi=UID=00000ce2bab78f39:T=1701651617:RT=1702340873:S=ALNI_MbP8LYnaonBenczWFditmUQm_kvgA',
            'If-None-Match': 'W/"31c7ac917838403d6c1869a7dbadecf5"',
            'Sec-Fetch-Dest': 'document',
            'Sec-Fetch-Mode': 'navigate',
            'Sec-Fetch-Site': 'none',
            'Sec-Fetch-User': '?1',
            'Upgrade-Insecure-Requests': '1',
            'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
            'sec-ch-ua': '"Not_A Brand";v="8", "Chromium";v="120", "Google Chrome";v="120"',
            'sec-ch-ua-mobile': '?0',
            'sec-ch-ua-platform': '"macOS"',
        }

        params = {
            'q': {book_title}
        }

        response = session.get('https://www.goodreads.com/search', params=params, cookies=cookies, headers=headers)
        print(f"scraping {book_title}")
        soup_doc = BeautifulSoup(response.text)
        # get all book matches 
        all_books = soup_doc.find_all('tr')

        found_first_occurence = False

        # now search for first author of each book and keep only the book that matches the first occurence of the correct author
            
        for book in all_books :
            test_author = book.select_one('.authorName').text
            test_author_clean = re.sub(r'[^A-Za-z0-9]', '', test_author).lower()
            if test_author_clean == re.sub(r'[^A-Za-z0-9]', '', author_name).lower() :
                book_name = book.select_one('.bookTitle').text
                author_link = book.select_one('.authorName')['href']
                author_name = book.select_one('.authorName').text
                rating = book.select_one('.minirating').text
                this_book = {
                'nyt_name' : book_title,    
                'good_reads_book' : book_name,
                'author' : author_name,
                'author_link' : author_link,
                'book_rating' : rating
                }
                found_first_occurence = True
                if found_first_occurence :
                    all_results.append(this_book)
                    break
                else :
                    continue      
    except Exception as e:
        print("Failed with", e.message)
        # If it fails, exit the loop
        break

scraping TICK TOCK
scraping THE GIRL WITH THE DRAGON TATTOO
scraping THE GIRL WHO PLAYED WITH FIRE
scraping THE GIRL WHO KICKED THE HORNET’S NEST
scraping WATER FOR ELEPHANTS
scraping THE CONFESSION
scraping CUTTING FOR STONE
scraping MARRYING DAISY BELLAMY
scraping THE HELP
scraping THE INNER CIRCLE
scraping STRATEGIC MOVES
scraping WILD MAN CREEK
scraping DEAD OR ALIVE
scraping SHADOWFEVER
scraping ROOM
scraping FATAL ERROR
scraping ALONE
scraping A DISCOVERY OF WITCHES
scraping THE SECRET SOLDIER
scraping DREAMS OF A DARK WARRIOR
scraping HER LAST LETTER
scraping HOTEL ON THE CORNER OF BITTER AND SWEET
scraping TREACHERY IN DEATH
scraping PALE DEMON
scraping GIDEON’S SWORD
scraping THIS SIDE OF THE GRAVE
scraping A CREED IN STONE CREEK
scraping HARVEST MOON
scraping SING YOU HOME
scraping THE WISE MAN'S FEAR
scraping RIVER MARKED
scraping MINDING FRANKIE
scraping LIVE WIRE
scraping THE JUNGLE
scraping LOVE YOU MORE
scraping THE LINCOLN LAWYER
scraping THE PARIS WIFE
scraping TOYS
sc

In [17]:
book_results= pd.DataFrame(all_results)
book_results.to_csv('goodreads-books.csv')